In [190]:
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm

from pathlib import Path
import re

TRAIN_DIR = Path("/home/lex/data/Spatial_Monitoring_and_Insect_Behavioural_Analysis_Dataset/YOLOv4_Training_and_Test_Dataset/training")
TEST_DIR = Path("/home/lex/data/Spatial_Monitoring_and_Insect_Behavioural_Analysis_Dataset/YOLOv4_Training_and_Test_Dataset/testing")
INSECT_TXT_REGEX = re.compile(r"insect_(\d+).txt")

In [191]:
# Get list of target classes (e.g. bee/wasp, flower, etc.) and corresponding zero-based index for that class
CLASSES = {
    0: "honeybee_vespidae", 
    1: "flower", 
    2: "syrphidae", 
    3: "lepidoptera"
}
classes_series = pd.Series(CLASSES, name="target_name")
classes_series

0    honeybee_vespidae
1               flower
2            syrphidae
3          lepidoptera
Name: target_name, dtype: object

In [192]:
# Get DataFrame where each row corresponds to a unique image+bounding box for every
# instance of an insect that occurs in the (full-resolution) training images
csv_columns = ["target", "centre_x", "centre_y", "width", "height"]
extra_columns = ["image"]
df = pd.DataFrame(columns=[*extra_columns, *csv_columns])

for file in TRAIN_DIR.iterdir():
    match = INSECT_TXT_REGEX.match(file.name)
    if match:
        row = pd.read_csv(file, sep=" ", names=csv_columns)
        
        # Get corresponding image for this insect 
        image_fp = file.with_suffix(".png")
        row["image"] = str(image_fp)
        df = pd.concat([df, row], ignore_index=True)

df.head()

,image,target,centre_x,centre_y,width,height
0,/home/lex/data/Spatial_Monitoring_and_Insect_B...,0,0.438305,0.512250,0.016172,0.033593
1,/home/lex/data/Spatial_Monitoring_and_Insect_B...,0,0.131612,0.412750,0.011568,0.035130
2,/home/lex/data/Spatial_Monitoring_and_Insect_B...,0,0.394792,0.306481,0.017708,0.025926
3,/home/lex/data/Spatial_Monitoring_and_Insect_B...,1,0.294792,0.488889,0.030208,0.055556
4,/home/lex/data/Spatial_Monitoring_and_Insect_B...,1,0.514062,0.241204,0.038542,0.065741


In [193]:
# Assume each image is the exact same resolution for simplicity. Now convert those fractional positions above to
# absolute pixel values (e.g. centre_x=0.44 --> 0.44*1920~=845)
image_fp = TRAIN_DIR / df.loc[0, "image"] # Just get first image as example
image = cv2.imread(str(image_fp))
height, width, num_channels = image.shape
print(f"Assuming every image is {width}x{height} resolution")

# Use image dimensions to convert fractional positions to absolute
df["centre_x"] = (df["centre_x"] * width).astype(int)
df["centre_y"] = (df["centre_y"] * height).astype(int)
df["width"] = (df["width"] * width).astype(int)
df["height"] = (df["height"] * height).astype(int)

df.head()

Assuming every image is 1920x1080 resolution


,image,target,centre_x,centre_y,width,height
0,/home/lex/data/Spatial_Monitoring_and_Insect_B...,0,841,553,31,36
1,/home/lex/data/Spatial_Monitoring_and_Insect_B...,0,252,445,22,37
2,/home/lex/data/Spatial_Monitoring_and_Insect_B...,0,758,330,33,28
3,/home/lex/data/Spatial_Monitoring_and_Insect_B...,1,566,528,57,60
4,/home/lex/data/Spatial_Monitoring_and_Insect_B...,1,986,260,74,71


In [194]:
# Add insect type as human-friendly string to dataframe just because
df = df.join(classes_series, on="target")
df.head()

,image,target,centre_x,centre_y,width,height,target_name
0,/home/lex/data/Spatial_Monitoring_and_Insect_B...,0,841,553,31,36,honeybee_vespidae
1,/home/lex/data/Spatial_Monitoring_and_Insect_B...,0,252,445,22,37,honeybee_vespidae
2,/home/lex/data/Spatial_Monitoring_and_Insect_B...,0,758,330,33,28,honeybee_vespidae
3,/home/lex/data/Spatial_Monitoring_and_Insect_B...,1,566,528,57,60,flower
4,/home/lex/data/Spatial_Monitoring_and_Insect_B...,1,986,260,74,71,flower


In [195]:
def crop_to_insect(row: pd.DataFrame, height_width:int=128):
    """Convert a row from our insect bounding boxes DataFrame into a cropped image."""
    # Note we're not actually using the original height/width of bounding boxes, 
    # we want something a little less precise and more consistent
    top_y = int(row["centre_y"] - 0.5 * height_width)
    bottom_y = int(row["centre_y"] + 0.5 * height_width)
    left_x = int(row["centre_x"] - 0.5 * height_width)
    right_x = int(row["centre_x"] + 0.5 * height_width)

    image = cv2.imread(row["image"])
    height, width, num_channels = image.shape

    # print(f"Before, top_y={top_y}, bottom_y={bottom_y}, left_x={left_x}, right_x={right_x}")
    # Check if our crop goes out of bounds
    pad_width = [[0, 0], [0, 0], [0, 0]] # padding width for (before, after) for each axis of image arrays
    if top_y < 0:
        pad_width[0][0] = abs(top_y)
        bottom_y += abs(top_y)
        top_y = 0
    if bottom_y > height:
        pad_width[0][1] = abs(bottom_y - height) + 1
    if left_x < 0:
        pad_width[1][0] = abs(left_x)
        right_x += abs(left_x)
        left_x = 0
    if right_x > width:
        pad_width[1][1] = abs(right_x - width) + 1
    # print(f"After, top_y={top_y}, bottom_y={bottom_y}, left_x={left_x}, right_x={right_x}")

    image_padded = np.pad(image, pad_width=pad_width, mode="symmetric")
    image_cropped = image_padded[top_y:bottom_y, left_x:right_x]

    return image_cropped

In [198]:
CLASSIFICATION_OUTPUT_DIR = Path("out/classification/")
CLASSIFICATION_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

for target in classes_series:
    target_dir = CLASSIFICATION_OUTPUT_DIR / target
    print(f"Creating directory for target if it doesn't already exist: {target_dir}")
    target_dir.mkdir(parents=True, exist_ok=True)

for index, row in tqdm(df.iterrows(), total=len(df)):
    row = row.squeeze()
    image_cropped = crop_to_insect(row)

    input_fp = Path(row["image"])
    image_stem = input_fp.stem
    image_suffix = input_fp.suffix
    target_name = row["target_name"]
    
    output_name = f"{image_stem}_{row['target']}_{row['centre_x']}_{row['centre_y']}{image_suffix}"
    output_fp = CLASSIFICATION_OUTPUT_DIR / target_name / output_name
    
    # print(output_fp)
    cv2.imwrite(str(output_fp), image_cropped)

Creating directory for target if it doesn't already exist: out/classification/honeybee_vespidae
Creating directory for target if it doesn't already exist: out/classification/flower
Creating directory for target if it doesn't already exist: out/classification/syrphidae
Creating directory for target if it doesn't already exist: out/classification/lepidoptera


 12%|█▏        | 1918/16550 [01:29<11:57, 20.40it/s]

In [ ]:
# # Rejig data's structure to work better with ImageFolder
# import shutil

# cropped_regex = re.compile(r"insect_(?P<image_index>\d+)_(?P<insect_type>\d)_(?P<x>\d+)_(?P<y>\d+)")

# count = {k: 0 for k in classes_series}
# num_files = len([f for f in CLASSIFICATION_OUTPUT_DIR.iterdir()])
# for file in tqdm(CLASSIFICATION_OUTPUT_DIR.iterdir(), total=num_files):
#     match = cropped_regex.match(file.name)
#     insect_type = int(match.group("insect_type"))
#     insect_type_name = classes_series.loc[insect_type]
#     destination_fp = Path(f"out/classification/{insect_type_name}/")
#     destination_fp.mkdir(parents=True, exist_ok=True)
#     # print(f"{file} --> {destination_fp}")
#     shutil.move(str(file), str(destination_fp))
#     # print(insect_type_name)
#     count[insect_type_name] += 1

# print(count)
